In [1]:
import anthropic
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import Optional
import sys
import os

# Get the current working directory (where the notebook is running)
cwd = os.getcwd()

# Add the backend directory to sys.path
sys.path.append(os.path.join(cwd, '..'))
from src.tools.utils import (
    formulate_pico, 
    run_parallel_mesh_queries, 
    formulate_esearch_query, 
    esearch_abstracts, 
    efetch_query_with_key,
    answer_from_abstracts,
    simple_research_pipeline
)

### As a reminder, this is the context that our LLM will start with

In [2]:
load_dotenv()

with open("../data/metrics_only.csv", "r", encoding="utf-8") as f:
    csv_string = f.read()

print(csv_string)

,biomarker.internalName,value,unit,deviationType,rangeType,visitDatetime
11,albumin,47.0,g/L,WITHIN_RANGE,LONGEVITY_GOOD,May 2024
80,albumin,45.0,g/L,WITHIN_RANGE,LONGEVITY_GOOD,April 2025
4,alp,81.0,U/L,WITHIN_RANGE,LONGEVITY_GOOD,May 2024
73,alp,113.0,U/L,WITHIN_RANGE,LONGEVITY_GOOD,April 2025
20,alt,42.0,U/L,WITHIN_RANGE,LONGEVITY_GOOD,May 2024
71,alt,38.0,U/L,WITHIN_RANGE,LONGEVITY_GOOD,April 2025
2,apolipoprotein-b,47.0,mg/dL,WITHIN_RANGE,LONGEVITY_GOOD,May 2024
62,apolipoprotein-b,80.0,mg/dL,WITHIN_RANGE,LONGEVITY_GOOD,April 2025
39,ast,48.0,U/L,ABOVE,MEDICAL_BORDERLINE,May 2024
70,ast,34.0,U/L,WITHIN_RANGE,LONGEVITY_GOOD,April 2025
47,bioavailable-testosterone,74.0,ng/dL,BELOW,MEDICAL_BORDERLINE,May 2024
50,bioavailable-testosterone,82.0,ng/dL,WITHIN_RANGE,MEDICAL_STANDARD,April 2025
42,biological-age,23.05,years old,WITHIN_RANGE,LONGEVITY_GOOD,May 2024
40,calcitonin,8.8,pg/mL,ABOVE,MEDICAL_BORDERLINE,May 2024
98,calcitonin,2.3,pg/mL,WITHIN_RANGE,LONGEVITY_GOOD,April 2025
34,cal

### We should mock some basic patient information, that the app will likely contain about a user:
- basic characteristics
- fitbit data

In [3]:
patient_info="""
PATIENT CHARACTERISTICS
sex: Male
age: 32
height: 177cm
weight: 72KG
"""

### Trying out Claude with some key prompts

In [4]:
# Load environment variables and initialize Anthropic client
load_dotenv()

# Create Anthropic client
client = anthropic.Anthropic(
    # The API key will be automatically read from the ANTHROPIC_API_KEY environment variable
)

In [5]:
system_prompt = """
You are Concierge, an AI doctor specialized in longevity, providing users with evidence-based insights and advice.
You answer questions in a clear manner, without omitting complex medical terminology, but providing detailed explanations.
"""
context = patient_info + "\n\n" + csv_string

question = """
Could I have a general summary of my blood test results ?
"""

user_prompt = f"""
INSTRUCTIONS: 
Given the CONTEXT, answer the given QUESTION. 


---- CONTEXT: ----
The user just recieved their updated blood test results, which are the following:

{context}




---- QUESTION: ----
{question}
"""

# Create the message structure for Anthropic
messages = [
    {
        "role": "user",
        "content": user_prompt
    }
]

In [6]:
# Call the Anthropic API
response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=4000,
    temperature=0,
    system=system_prompt,
    messages=messages
)

# Print the response
print(response.content[0].text)

# Blood Test Results Summary

## Overall Assessment
Your April 2025 blood test results show significant improvements compared to May 2024, with most parameters now within optimal ranges. Your biological age was measured at 23.05 years (9 years younger than your chronological age), which is excellent.

## Key Improvements
- **Hormones**: Your testosterone levels (both free and bioavailable) have improved from borderline low to normal range
- **Thyroid function**: T3 and T4 have normalized from previously low values
- **White blood cells**: WBC count has improved from below range to optimal
- **Liver function**: AST has normalized from previously elevated levels
- **Lymphocytes**: Have increased from below range to optimal levels
- **Calcitonin**: Has decreased from elevated to normal range

## Areas of Concern
1. **HDL Cholesterol**: Has decreased significantly from 80 mg/dL to 38 mg/dL (below optimal range)
2. **Lipoprotein(a)**: Remains critically elevated at 750 mg/L (increased from 

### The good

- LLM understood the chronological aspect of the data, and compared 2025 to 2024 results. 
- It put an emphasis on biological age despite it being presented among other biomarkers, which means it understood the business context it's been put in (wellness and longevity)

### The  bad
- The LLM's insights feel superficial, there is no link between different biomarker results
- Of course, recommendations are made without articles being cited, which we must correct

# Attempt #2 : using structured output to integrate pubmed search

- Right now, pubmed_search is just a dummy function, we're testing intelligent function calling first. Then, we'll incorporate a real function instead

In [7]:
# Define the PubMedSearch tool as a function that will be used in Claude's tool definition
def pubmed_search(queris: str) -> str:
    """Use the pubmed API to obtain information on a certain query"""
    return "foo"

# Define the tool in the format Anthropic expects
pubmed_tool = {
    "name": "pubmed_search",
    "description": "A LLM chain that will intelligently run one or multiple searches for you, returning relevant articles as well as key findings from each.",
    "input_schema": {
        "type": "object",
        "properties": {
            "queries": {
                "type": "array",
                "items": {"type": "string"},
                "description": "Here, you may include a list of research questions that you would like answered, in the context of your patient. Make the queries as specific as possible, with the objective of finding hyper-relevant solutions to the patient's problem."
            }
        },
        "required": ["query"]
    }
}

# Create a dictionary to map tool calls to actual functions
tool_map = {
    "pubmed_search": pubmed_search
}

In [8]:
system_prompt = """
You are Concierge, an AI doctor specialized in longevity, providing users with evidence-based insights and advice.
You answer questions in a clear manner, without omitting complex medical terminology, but providing detailed explanations.
You never give medical information that isn't backed by sources
You are a step-by step reasoning model, giving continual feedback on your reasoning to the user using structured output
"""

context = patient_info + "\n\n" + csv_string

question = """
What are the most important things I should currently do to improve my health ?
"""

user_prompt = f"""
INSTRUCTIONS: 
Given the CONTEXT, answer the given QUESTION. 
Never give any information that isn't backed by clear sources. If you need sources, use the pubmed_search tool
If you decide to use the tool, make sure to also explain what you're going to be searching and why.


---- CONTEXT: ----
The user just recieved their updated blood test results, which are the following:

{context}




---- QUESTION: ----
{question}
"""

# Create the message structure for Anthropic with tools
messages = [
    {
        "role": "user",
        "content": user_prompt
    }
]

In [9]:
# Call the Anthropic API with tools
response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=4000,
    temperature=0,
    messages=messages,
    system=system_prompt,
    tools=[pubmed_tool]
)

In [37]:
assistant_response = response.content[0].text
print(assistant_response, "\n\n")
tool_use = response.content[-1]
print(response.content[-1])

new_messages = messages.copy()
new_messages.append(
    {"role":"assistant","content":assistant_response}
)

I'll analyze your blood test results from May 2024 and April 2025 to identify the most important areas for health improvement. Let me first identify the key concerns in your most recent results (April 2025).

## Initial Analysis of Your Blood Tests

Looking at your April 2025 results, I notice several important areas that need attention:

1. **HDL Cholesterol**: 38 mg/dL (BELOW MEDICAL BORDERLINE) - This is concerning as HDL is the "good" cholesterol
2. **Lipoprotein(a)**: 750 mg/L (ABOVE MEDICAL CRITICAL) - This is significantly elevated
3. **Vitamin D**: 23 ng/mL (BELOW MEDICAL BORDERLINE) - This remains low in both tests

Let me search for evidence-based recommendations for these specific issues. 


ToolUseBlock(id='toolu_01HNW7tptgncsUT3Wuecyzi8', input={'queries': ['Strategies to increase HDL cholesterol in adult males', 'Lipoprotein(a) reduction interventions and cardiovascular risk', 'Vitamin D supplementation guidelines and optimal levels for health']}, name='pubmed_search', ty

## Enter the PubMed Search tool, as demonstrated in notebook 4

In [11]:
import src.tools.utils as ut
import src.tools.pubmed_search as ps

# Create a dictionary to map tool calls to actual functions
tool_map = {
    "pubmed_search": ps.run_parallel_research_for_queries
}

In [12]:


tool_calls = ut.search_tool_calls(response)
if tool_calls['has_tool_use'] == True:
    for tool in tool_calls['tools']:
        function_name = tool['tool_name']
        if function_name == "pubmed_search":
            params = tool['tool_input']
            res = await ps.run_parallel_research_for_queries(params, context)

✅ Successfully parsed 15 articles from PubMed
✅ Successfully parsed 15 articles from PubMed
✅ Successfully parsed 14 articles from PubMed


In [41]:
new_new_messages = new_messages.copy()
new_new_messages.append(
    {
        "role":"user", 
        "content":f"Here are the results of the PubMed research tool : {res} \n\n Formulate your final answer, **making sure to include all relevant article links**"
    },
)

In [42]:
# Call the Anthropic API
response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=4000,
    temperature=0,
    system=system_prompt,
    messages=new_new_messages
)

# Print the response
print(response.content[0].text)

# Key Health Improvement Recommendations Based on Your Blood Tests

Based on your April 2025 blood test results, I've identified three critical areas that need attention to improve your overall health and longevity. Let me provide evidence-based recommendations for each.

## 1. Improve HDL Cholesterol Levels

Your HDL cholesterol has dropped significantly from 80 mg/dL in May 2024 to 38 mg/dL in April 2025, which is below the medical borderline. HDL is considered "good" cholesterol that helps remove other forms of cholesterol from your bloodstream.

**Recommendations:**
- **Increase physical activity**: Implement a structured exercise program with both aerobic exercise (30-40 minutes, 4-5 times weekly) and resistance training (2-3 sessions weekly)
- **Adopt an anti-inflammatory diet**: Focus on foods rich in omega-3 fatty acids (fatty fish, walnuts, flaxseeds)
- **Increase consumption of healthy fats**: Include monounsaturated fats from olive oil, avocados, and nuts
- **Increase legume